In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
def idFinder(idList):
    for id in idList:
        if len(id)>4:
            return id
    # if not found
    return 'noID'

def get_bs(page_no):
    r=requests.get('https://www.therugshopuk.co.uk/rugs-by-type/rugs.html?p='+str(page_no))
    rugSoup=bs(r.text,features='html.parser')
    return rugSoup

def get_items(rugSoup):
    items=[]
    for item in rugSoup.find_all('li',{'class':'item product product-item'}):
        try:
            # just add another item that you want to scrap
            miniItem={
                'imageLink':item.img.get('src'),
                'rugName':item.img.get('alt'),
                'rugId':idFinder((item.find('div',{'class':'sku-bg'}).get_text()).split(' ')), # can modify bit with some function to get only neat ID no
                'rugLink':item.find('a',{'class':'product photo product-item-photo'})['href'],
                'rugPrice':item.find('span',{'class':'price'}).text
            }
        except Exception as e:
            print('Here we have this problem ------ %s'%e)
            pass
        items.append(miniItem)
    return items

In [ ]:
all_items=[]
userPages=int(input('How many pages to scrape -->'))
pagesToScrape=min(userPages,999)+1 # see how range() methos works
for page in range(1,pagesToScrape): # as page number starts from 1
    print('scrapping through page .... ',page)
    currSoup=get_bs(page)
    if not currSoup.find('div',{'class':'pages'}).find('a',{'class':'action next'}):
        print('we have reached to the end')
        break
    else:
        all_items.append(get_items(currSoup))
    print('all asked pages scrapped')

def flattner(totalList):
    return [rugItem for pageItem in all_items for rugItem in pageItem]

rugData=pd.DataFrame(flattner(all_items))
#rugData.to_csv('rug-shop-uk.csv',index=False) #<------ uncomment to store into CSVb